# Part 2: Creating your first Experiment in CRIPT 
---

In this notebook we will create document our first experiment, and all the associated data. The following example is the anionic polymerization of styrene with secBuLi in a mixture of THF and toluene. The reaction is then quenched with butanol and precipitated into methanol to obtain the final product, polystyrene.

![Polystyrene_synthesis](../Jupyter_notebook/supporting_files/polystyrene_synthesis.svg)


---

Before diving directly into this examples we will start with the basics: importing cript module, and connecting to the CRIPT database.

<u>Action:</u> Replace 'user = ###@###.##' with the email address you used for your `User node` or user id. Then run the cell. <span style="color:gray">(You can run a cell by either clicking on it and hitting the 'play/run' button in the toolbar or click the cell and hit "shift" + "enter".)</span>

<u>Output:</u> You should see "Import sucessful!", "Connection to database 'test' successful.", "you logged in sucessfully." 

In [1]:
import cript as C
print("Import successful!")

# Connect to database
db_username = "DW_cript"
db_password = "YXMaoE1"
db_project = "cript_testing"
db_database = "test"
user = "johndoe13@cript.edu"  # Put your email or uid here
db = C.CriptDB(db_username, db_password, db_project, db_database, user)

Import successful!
Connection to database 'test' successful.



<span style="color:SteelBlue"> This previous block of code is the standard code you should type first for all future projects. </span> 

---

## Collection and Experiment Nodes

As we look to create our first expermient there are two orginizational nodes that CRIPT provides: `C.Collection()` and `C.Experiment`.

In [9]:
collection = C.Collection("Anionic Polymerizations")
print(collection)

{
  "class_": "Collection",
  "model_version": "0.0.1",
  "name": "Anionic Polymerizations"
}


Now that we create the collection we will want to save it and pick a group to own it. So lets view what groups we are apart of again. To do that, we can use the fact that we are logged in and just call `db.user`.

In [14]:
groups = db.view(cript.Groups)

AttributeError: module 'cript' has no attribute 'Groups'

We can now see the groups we joined in part 1. We will put this collection into "tutorial_*****". To do this, we just need to pass the node want to save and then the group we want to save it to with it. 

In [ ]:
db.save(collection, groups[0])

---

Now we have a collection made, let make our `Experiment node`.


In [11]:
expt = C.Experiment("Anionic Polymerization of Styrene with SecBuLi")
print(expt)

{
  "class_": "Experiment",
  "model_version": "0.0.1",
  "name": "Anionic Polymerization of Styrene with SecBuLi"
}


To view our collections of our user node

In [ ]:
coll = db.view(cript.Collections)

In [ ]:
db.save(expt, coll[0])

---
---

## Materal Node

### Creating a Material Node


In [ ]:
help(cript.Material.__init__)

we see that the only required paramter is 'identifier'. Looking at the :param we see it is c_identifier meaning it is a CRIPT object.

In [ ]:
help(cript.Identifiers.__init__)

The minimuim requirement is the preferred_name, but its best to put as much as possible.

In [18]:
iden = [cript.Identifiers(
    preferred_name="styrene",
    names=["vinylbenzene", "phenylethylene", "ethenylbenzene"],
    chem_formula="C8H8",
    smiles="C=Cc1ccccc1",
    cas="100-42-5",
    pubchem_cid="7501",
    inchi_key="PPBRXRYQALVLMV-UHFFFAOYSA-N"
)]

print(iden[0])

In [27]:
isinstance(iden[0], tuple(C.cript_types.values()))

True

In [ ]:
mat_styrene = cript.Material(
    identifier=iden,
    keywords=["styrene"],
    storage=[
        cript.Cond(key="temp", value=-20, unit=Unit("degC")),
        cript.Cond(key="atm", value="argon")
    ]
    )

In [ ]:
 unit=Unit("degC")

In [ ]:
help(unit)

In [ ]:
# add properties
prop = [cript.Prop(mat_id=0, key="phase", value="liquid"),
        cript.Prop(mat_id=0, key="color", value="colorless"),
        cript.Prop(mat_id=0, key="mw", value=104.15, unit=Unit("g/mol"), method="prescribed"),
        cript.Prop(mat_id=0, key="density", value=0.906, unit=Unit("g/ml"),
                   conditions=[cript.Cond(key="temp", value=25, unit=Unit("degC"))]
                   ),
        cript.Prop(mat_id=0, key="bp", value=145, unit=Unit("degC"),
                   conditions=[cript.Cond(key="pressure", value=1, unit=Unit("atm"))]
                   ),
        cript.Prop(mat_id=0, key="mp", value=-30, unit=Unit("degC"),
                   conditions=[cript.Cond(key="pressure", value=1, unit=Unit("bar"))]
                   ),
        cript.Prop(mat_id=0, key="solubility", value=0.3, unit=Unit("g/mol"),
                   conditions=[
                             cript.Cond(key="temp", value=25, unit=Unit("degC")),
                             cript.Cond(key="solvent", value="water", unit=Unit("bar"))
                         ]
                   ),
        ]

node.properties = prop

In [ ]:
saves both

In [ ]:
db.save(mat_styrene, expt)


---

### Adding a Existing Material Node


In [ ]:
query = {
    "field": "identifier.name",
    "type" : "$regex",
    "value": "toluene"
}
tol_query = db.view(cript.Material, query)
query = {
    "field": "identifier.name",
    "type" : "$regex",
    "value": "thf"
}
thf_query = db.view(cript.Material, query)

In [ ]:
tol_query[0]

In [ ]:
expt.c_material = [tol_query[0], thf_query[0]]

---

### Copying an existing Material Node

In [ ]:
query = {
    "field": "identifier.name",
    "type" : "$regex",
    "value": "BuLi"
}
BuLi_query = db.view(cript.Material, query)

In [ ]:
BuLi_query[0]

In [ ]:
BuLi_node = cript.copy(BuLi_query[0])
BuLi_node.properties["conc"] = "1.2"

In [ ]:
db.save(BuLi_node, expt)

In [8]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  '__dir__',
  '__dir__()',
  '__dir__',
  'globals()',
  'from cript import *',
  'globals()',
  'import cript',
  'globals()'],
 '_oh': {4: {...}, 6: {...}},
 '_dh': ['C:\\Users\\nicep\\Desktop\\Reseach_Post\\Case_studies\\cript\\examples\\Jupyter_notebook'],
 'In': ['',
  '__dir__',
  '__dir__()',
  '__dir__',
  'globals()',
  'from cript import *',
  'globals()',
  'import cript',
  'globals()'],
 'Out': {4: {...}, 6: {...}},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x000001F777418DF0>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x1f777418340>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x1f777418340>,
 '_': {...},
 '__': {

In [7]:
import cript

In [ ]:
db.update(expt)


---
---

## Key Takeaways

* You have 

---
---

### What's next?
